In [1]:
import pandas as pd
import json

In [2]:
with open('datasets/mmlu_ro/ro_test.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)


In [3]:
df.head()

,instruction,option_a,option_b,option_c,option_d,answer,id
0,Pentru care dintre cele două scenarii face per...,"Greșit, Greșit","Greșit, Nu greșit","Nu greșit, Greșit","Nu greșit, Nu greșit",B,moral_scenarios/test/106
1,Grupul Națiunii Islamice a apelat la:,Imigranți din a doua generație originari din s...,Americani albi care au vrut să se convertească...,Afro-americani care s-au simțit excluși din «t...,Afro-caraibieni care trăiau în zonele centrale...,C,sociology/test/49
2,Care parte a spectrului electromagnetic are ce...,Razele gamma,Raze X,Undele radio,Undele microundelor,A,miscellaneous/test/535
3,Scopul principal al declarației de activitate ...,Furnizorilor de resurse.,Managerilor.,Beneficiarilor.,Organelor de reglementare a statului.,A,professional_accounting/test/251
4,O femeie era proprietara unei ferme in proprie...,"femeia era proprietarul legitim si, prin urmar...","femeia avea dreptul la posesie, deoarece poses...","barbatul avea dreptul la posesie, deoarece pos...","barbatul avea dreptul la posesie, datorita pri...",D,professional_law/test/879


In [4]:
llama_3_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Sunteți un asistent util, respectuos și onest. Veți primi o instrucțiune și 4 variante de răspuns. O singură variantă este corectă. 
Răspundeți doar cu litera corespunzătoare variantei corecte.<|eot_id|><|start_header_id|>user<|end_header_id|>

{instruction}
A. {option_a}
B. {option_b}
C. {option_c}
D. {option_d}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

llama_2_prompt = """<s>[INST] <<SYS>>
Sunteți un asistent util, respectuos și onest. Dacă o întrebare nu are niciun sens sau nu este coerentă din punct de vedere factual, explicați de ce în loc să răspundeți la ceva incorect. 
Dacă nu știți răspunsul la o întrebare, vă rugăm să nu împărtășiți informaţii false. Trebuie sa răspundeți doar în limba română.
<</SYS>>

{instruction}
A. {instruction}
B. {instruction}
C. {instruction}
D. {instruction} [/INST]
"""

alpaca_prompt = """Mai jos este prezentată o instrucțiune care descrie o sarcină, însoțită de o intrare care oferă un context suplimentar. Scrieți un răspuns care să completeze în mod corespunzător cererea. 
Trebuie să răspundeți numai în limba română.

### Instrucțiune:
{instruction}

### Intrare:
A. {instruction}
B. {instruction}
C. {instruction}
D. {instruction}

### Răspuns:
"""

In [6]:
unique_ids = df['id'].unique()
print(unique_ids)

['moral_scenarios/test/106' 'sociology/test/49' 'miscellaneous/test/535'
 ... 'professional_law/test/125' 'international_law/test/9'
 'anatomy/test/66']


In [95]:
categories = df['id'].str.split('/')
unique_categories = categories.str[0].unique()
print(unique_categories)


['moral_scenarios' 'sociology' 'miscellaneous' 'professional_accounting'
 'professional_law' 'human_aging' 'astronomy' 'high_school_mathematics'
 'college_medicine' 'high_school_psychology' 'logical_fallacies'
 'philosophy' 'marketing' 'conceptual_physics' 'professional_psychology'
 'high_school_microeconomics' 'elementary_mathematics'
 'high_school_us_history' 'prehistory' 'high_school_macroeconomics'
 'world_religions' 'electrical_engineering' 'virology' 'jurisprudence'
 'high_school_statistics' 'high_school_european_history'
 'high_school_government_and_politics' 'computer_security'
 'high_school_computer_science' 'high_school_physics' 'international_law'
 'high_school_biology' 'business_ethics' 'high_school_world_history'
 'machine_learning' 'formal_logic' 'clinical_knowledge' 'college_physics'
 'high_school_geography' 'moral_disputes' 'high_school_chemistry'
 'human_sexuality' 'nutrition' 'security_studies' 'college_chemistry'
 'abstract_algebra' 'college_mathematics' 'college_bio

In [5]:
import requests

def generate_response(prompt):
    url = "http://10.198.110.23:8000/generate"  # Replace "ADDRESS" with the actual address

    payload = {
        "prompt": prompt,
        "n": 1,
        "temperature": 0.95,
        "max_tokens": 1024
    }

    response = requests.post(url, json=payload)

    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code)
        return None

def get_chosen_answer(row):
    response_data = generate_response(row['formatted_prompt'])
    if response_data:
        answer = response_data['text'][0].split('\n')[-1]
        return answer
    else:
        return None


In [97]:
pd.set_option('display.max_colwidth', None)

# Increase maximum number of rows displayed
pd.set_option('display.max_rows', None)

In [6]:
df['formatted_prompt'] = df.apply(lambda row: llama_2_prompt.format(**row).strip(), axis=1)

df['formatted_prompt'][1]

'<s>[INST] <<SYS>>\nSunteți un asistent util, respectuos și onest. Dacă o întrebare nu are niciun sens sau nu este coerentă din punct de vedere factual, explicați de ce în loc să răspundeți la ceva incorect. \nDacă nu știți răspunsul la o întrebare, vă rugăm să nu împărtășiți informaţii false. Trebuie sa răspundeți doar în limba română.\n<</SYS>>\n\nGrupul Națiunii Islamice a apelat la:\nA. Grupul Națiunii Islamice a apelat la:\nB. Grupul Națiunii Islamice a apelat la:\nC. Grupul Națiunii Islamice a apelat la:\nD. Grupul Națiunii Islamice a apelat la: [/INST]'

In [99]:
formatted_prompt = df['formatted_prompt'][1]

formatted_prompt

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nSunteți un asistent util, respectuos și onest. Veți primi o instrucțiune și 4 variante de răspuns. O singură variantă este corectă. Răspundeți doar cu litera corespunzătoare variantei corecte.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGrupul Națiunii Islamice a apelat la:\nA. Imigranți din a doua generație originari din subcontinentul asiatic născuți în Marea Britanie\nB. Americani albi care au vrut să se convertească la islam\nC. Afro-americani care s-au simțit excluși din «toporul etnic» din SUA\nD. Afro-caraibieni care trăiau în zonele centrale ale orașelor și aveau o cultură distinctivă a tinerilor<|eot_id|><|start_header_id|>assistant<|end_header_id|>'

In [100]:
df['chosen_answer'] = df.apply(lambda row: get_chosen_answer(row), axis=1)


ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [87]:
df['chosen_answer'].head()

0                                         D. 0.33
1    B. O doză de încărcare de clorură de potasiu
2                         Răspunsul este C. bine.
3                                               A
Name: chosen_answer, dtype: object

In [74]:
new_df = df[['instruction', 'option_a', 'option_b', 'option_c', 'option_d', 'answer', 'chosen_answer']].copy()
new_df.rename(columns={'answer': 'correct_answer'}, inplace=True)

In [75]:
new_df.head()

,instruction,option_a,option_b,option_c,option_d,correct_answer,chosen_answer
0,Să se ia în considerare următorul model AR(1) cu tulburări cu medie zero şi varianţă unitară\n\nyt = 0.2 + 0.4 yt-1 + ut\n\n(Media condiţionată) a lui y va fi dată de,0.2,0.4,0.5,0.33,D,A. 0.2
1,"Un bărbat în vârstă de 42 de ani vine la cabinet pentru evaluarea preoperatorie înainte de a suferi o adrenalectomie programată în 2 săptămâni. Cu o lună în urmă, el a primit îngrijiri în departamentul de urgență pentru dureri la flancul drept după o coliziune auto. În acel moment, tensiunea arterială a fost de 160/100 mm Hg și CT-ul la abdomen a arătat o masă incidentală a glandei suprarenale stângi de 10 cm. Rezultatele studiilor de laborator, inclusiv hemograma, concentrațiile serice de electroliți și testele funcției hepatice, erau în limite normale. Pacientul în rest a fost sănătos și nu i s-a spus niciodată că are tensiune arterială crescută. Nu ia medicamente. O vizită de urmărire în cabinet cu 2 săptămâni în urmă a dezvăluit concentrări crescute de normetanefrină și metanefrină urinare și de aldosteron plasmatic. Pacientul a fost informat despre necesitatea unei intervenții chirurgicale de către un chirurg, care a recomandat adrenalectomia. Astăzi, semne vitale sunt temperatura 36,6°C (97,9°F), puls 100/min, respirații 14/min și tensiune arterială de 170/95 mm Hg. Examenul fizic nu dezvăluie constatări semnificative. Pregătirea preoperatorie inițială ar trebui să includă tratamentul cu una dintre următoarele opțiuni?",Labetalol,O doză de încărcare de clorură de potasiu,Nifedipină,Fenoxibenzamină,D,D. Fenoxibenzamina
2,"Conform utilitarismului ideal al lui Moore, acțiunea corectă este cea care aduce cele mai multe:",plăceri.,fericire.,bine.,virtute.,C,C. bine
3,Care dintre următoarele nu este inclus în PIB-ul SUA?,Militarul SUA deschide o nouă bază într-o țară străină cu 1000 de personal militar american.,Consumatorii japonezi cumpără mii de CD-uri produse în Statele Unite.,Un cântăreț pop american susține un concert cu casa închisă la Paris.,O producție teatrală franceză face turnee prin zeci de orașe americane.,C,D. O producție teatrală franceză face turnee prin zeci de orașe americane.
